In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
import math
from sklearn.model_selection import train_test_split
import re
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from dateutil.rrule import rrule, DAILY 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score 
import math 
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import FunctionTransformer 
from sklearn.compose import ColumnTransformer 
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import re
from sklearn.preprocessing import OrdinalEncoder

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\xinrui
[nltk_data]     zhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\xinrui
[nltk_data]     zhan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def remove_stop_words(data):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(data) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return ' '.join(filtered_sentence)

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        data = str.replace(data, i, ' ')
    return data

def remove_apostrophe(data):
    return str.replace(data, "'", "")

def remove_single_characters(data):
    new_text = ""
    words = word_tokenize(data)
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def stemming(data):
    return PorterStemmer().stem(data)

In [4]:
def preprocess(data):
    data = str(data)
    data = data.lower()
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    return data

In [9]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

C:\Users\xinrui zhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
def p_label(x):
    if x > 700:
        return "high"
    elif x>500:
        return 'less high'
    elif x>100:
        return "middle"
    else:
        return 'low'

In [11]:
train_df.columns

Index(['id', 'name', 'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'host_id', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
       'host_listings_count', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'country_code', 'country', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_ch

In [61]:
train_df.transit.head()

0                            Subway: 2,3,4,5,A,C,B,Q,G
1    PUBLIC TRANSPORTATION: Conveniently near all p...
2                                                  NaN
3                                                  NaN
4    Super convenient to almost all subway lines. A...
Name: transit, dtype: object

In [62]:
np.mean(train_df.review_scores_cleanliness.isnull())

0.22896416005724848

In [55]:
train_df.square_feet.value_counts()

1000.0    27
500.0     27
0.0       22
700.0     20
800.0     15
400.0     14
1200.0    13
900.0     13
750.0     13
600.0     12
450.0     11
850.0     10
1500.0     8
650.0      8
550.0      7
950.0      7
2000.0     6
1800.0     6
350.0      5
250.0      5
1100.0     4
200.0      4
1600.0     4
300.0      4
1300.0     3
100.0      3
150.0      3
140.0      3
1400.0     2
3600.0     2
          ..
484.0      1
96.0       1
210.0      1
790.0      1
915.0      1
330.0      1
720.0      1
240.0      1
245.0      1
17.0       1
2160.0     1
175.0      1
5000.0     1
1090.0     1
982.0      1
875.0      1
12.0       1
1150.0     1
1010.0     1
220.0      1
925.0      1
135.0      1
410.0      1
2200.0     1
269.0      1
665.0      1
1750.0     1
725.0      1
625.0      1
540.0      1
Name: square_feet, Length: 83, dtype: int64

In [63]:
train_df.groupby('review_scores_cleanliness')['price'].mean()

review_scores_cleanliness
2.0     126.340206
3.0      85.000000
4.0     119.275229
5.0     120.943662
6.0     127.127563
7.0     124.332298
8.0     121.365693
9.0     135.563164
10.0    145.688609
Name: price, dtype: float64

In [12]:
train_df.extra_people.apply(lambda x: int(x[:-3].replace('$', ''))).value_counts()

0      18007
25      2799
20      2528
10      2018
50      1800
15      1538
30      1240
40       569
35       510
100      333
5        324
75       253
45       181
60       126
12        82
7         78
80        63
55        60
18        58
29        57
19        52
39        51
300       46
150       45
8         41
70        35
49        32
9         32
200       32
65        31
       ...  
57         2
130        2
105        2
129        2
110        2
31         2
52         2
115        2
145        2
67         1
285        1
225        1
189        1
190        1
68         1
51         1
61         1
199        1
89         1
83         1
88         1
56         1
119        1
76         1
87         1
149        1
143        1
175        1
176        1
41         1
Name: extra_people, Length: 98, dtype: int64

In [245]:
train_df.amenities = train_df.amenities.apply(preprocess).apply(lambda x: word_tokenize(x))

In [258]:
temp = {}
for lst in train_df.amenities:
    for item in lst:
        if item not in temp.keys():
            temp[item] = 1
        else:
            temp[item]+=1

In [282]:
values = np.array(list(temp.values()))
keys = np.array(list(temp.keys()))
ind = (values > 1000) & (values < 33000)
display(len(keys[ind]))
related = keys[ind]

131

In [286]:
def one_hot(lst):
    temp = np.zeros(131)
    for i in range(len(related)):
        if related[i] in lst:
            temp[i] = 1
    return temp

In [277]:
train_df.shape

(33538, 65)

In [244]:
word_tokenize(preprocess(train_df.amenities[0]))

['wifi',
 'air',
 'conditioning',
 'kitchen',
 'gym',
 'breakfast',
 'elevator',
 'heating',
 'washer',
 'dryer',
 'smoke',
 'detector',
 'carbon',
 'monoxide',
 'detector',
 'first',
 'aid',
 'kit',
 'safety',
 'card',
 'fire',
 'extinguisher',
 'essentials',
 'shampoo',
 'hangers',
 'hair',
 'dryer',
 'iron',
 'laptop',
 'friendly',
 'workspace',
 'self',
 'check',
 'building',
 'staff',
 'hot',
 'water',
 'luggage',
 'dropoff',
 'allow']

In [74]:
summary_df = pd.DataFrame()
summary_df['transit'] = train_df.transit.apply(preprocess) 
summary_df['label'] = train_df.price.apply(lambda x: p_label(x))
words_in_bin = summary_df.groupby('label')['transit']

In [75]:
words_dct = {}
for label, grp in words_in_bin:
    grp = grp.to_list()
    #display(type(grp[0]))
    para = ' '.join(grp)
    words_dct[label] = para

In [76]:
words_dct.keys()
words = list(words_dct.values())

In [77]:
df = {}
for i in range(len(words)):
    sentence = words[i]
    tokens = word_tokenize(sentence)
    for w in tokens:
        try:
            df[w].add(i)
        except:
            df[w] = {i}

In [78]:
for i in df.keys():
    df[i] = len(df[i])

In [79]:
total = df.keys()

In [80]:
len(total)

10825

In [81]:
tf = {}
for i in words_dct.keys():
    para = words_dct[i]
    tokens = word_tokenize(para)
    temp = {}
    for token in tokens:
        try:
            temp[token] += 1
        except:
            temp[token] = 1
    tf[i] = temp

In [98]:
top_tens = []
for i in words_dct.keys():
    tf_idf = []
    for word in tf[i].keys():
        tf_ = tf[i][word]
        df_ = df[word]
        idf_ = (5)/(df_+1)
        tf_idf_ = math.log(idf_)*tf_
        tf_idf.append(tf_idf_)
    temp = pd.DataFrame()
    temp['word'] = tf[i].keys()
    temp['tf_idf'] = tf_idf
    top_ten = list(temp.sort_values('tf_idf', ascending=False)['word'][:20])
    top_tens.append(top_ten)

In [99]:
top_tens = pd.DataFrame(top_tens)
top_tens

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,wyndham,bentley,•subway,rockette,highline,sampling,headquarters,kicks,going,concierge,bathing,m55,•34,regis,suit,•nyc,provid,166,figur,meadowland
1,wyndham,•34,•nyc,¬–,35,50,quick,rockette,sampling,kicks,depending,circle,headquarters,fq,em6,m55,tribecca,lispendard,hellgate,volvo
2,utica,along,road,transfer,myrtle,parkway,quick,av,35,morgan,halsey,nostrand,mall,graham,blvd,coming,rail,mta,wyckoff,b38
3,along,m15,av,south,quick,parkway,extremely,road,share,transfer,coming,35,1st,nassau,utica,graham,locations,apart,nostrand,making


In [173]:
def extract_sum(summary):
    words = word_tokenize(summary)
    if 'bentley' in words:
        return 1
    else:
        return 0

In [161]:
train_df = pd.read_csv('train.csv')
after_pre =train_df.transit.apply(preprocess)

In [174]:
train_df.loc[:, 'transit'] = after_pre.apply(extract_sum)
train_df.groupby('transit')['price'].size()

transit
0    33537
1        1
Name: price, dtype: int64

In [175]:
train_df.groupby('transit')['price'].mean()

transit
0     145.151206
1    1020.000000
Name: price, dtype: float64

In [101]:
test_df = pd.read_csv('test.csv')
t_after_pre =test_df.summary.apply(preprocess)

In [171]:
test_df.loc[:, 'transit'] = t_after_pre.apply(extract_sum)
test_df.groupby('transit')['beds'].size()

description
0    17209
1      128
Name: beds, dtype: int64